In [27]:
import pandas as pd
import ast
from sklearn.metrics import classification_report, accuracy_score
import numpy

In [28]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
import json

with open('/content/drive/My Drive/MedNLI_Clinical_Inference/mli_train_v1.jsonl', 'r') as json_file:
    json_list_train = list(json_file)

with open('/content/drive/My Drive/MedNLI_Clinical_Inference/mli_dev_v1.jsonl', 'r') as json_file:
    json_list_val = list(json_file)

with open('/content/drive/My Drive/MedNLI_Clinical_Inference/mli_test_v1.jsonl', 'r') as json_file:
    json_list_test = list(json_file)

In [4]:
sent1_train = []
sent2_train = []
gl_train = []

for i in range(len(json_list_train)):
    a = ast.literal_eval(json_list_train[i])
    sent1_train.append(a['sentence1'])
    sent2_train.append(a['sentence2'])
    gl_train.append(a['gold_label'])

sent1_val = []
sent2_val = []
gl_val = []

for i in range(len(json_list_val)):
    a = ast.literal_eval(json_list_val[i])
    sent1_val.append(a['sentence1'])
    sent2_val.append(a['sentence2'])
    gl_val.append(a['gold_label'])

sent1_test = []
sent2_test = []
gl_test = []

for i in range(len(json_list_test)):
    a = ast.literal_eval(json_list_test[i])
    sent1_test.append(a['sentence1'])
    sent2_test.append(a['sentence2'])
    gl_test.append(a['gold_label'])

In [5]:
data_train = pd.DataFrame()
data_train['sentence1'] = sent1_train
data_train['sentence2'] = sent2_train
data_train['gold_label'] = gl_train

data_val = pd.DataFrame()
data_val['sentence1'] = sent1_val
data_val['sentence2'] = sent2_val
data_val['gold_label'] = gl_val

data_test = pd.DataFrame()
data_test['sentence1'] = sent1_test
data_test['sentence2'] = sent2_test
data_test['gold_label'] = gl_test

In [6]:
print(data_train.info())
print(data_val.info())
print(data_test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11232 entries, 0 to 11231
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   sentence1   11232 non-null  object
 1   sentence2   11232 non-null  object
 2   gold_label  11232 non-null  object
dtypes: object(3)
memory usage: 263.4+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1395 entries, 0 to 1394
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   sentence1   1395 non-null   object
 1   sentence2   1395 non-null   object
 2   gold_label  1395 non-null   object
dtypes: object(3)
memory usage: 32.8+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1422 entries, 0 to 1421
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   sentence1   1422 non-null   object
 1   sentence2   1422 non-null   object
 2   gold_label  1422 non-n

In [7]:
data_train['gold_label'].value_counts(normalize = True)

entailment       0.333333
contradiction    0.333333
neutral          0.333333
Name: gold_label, dtype: float64

In [8]:
data_train['combined_sent'] = data_train['sentence1']+ data_train['sentence2']
data_val['combined_sent'] = data_val['sentence1']+ data_val['sentence2']
data_test['combined_sent'] = data_test['sentence1']+ data_test['sentence2']

In [9]:
import re

def split_string(text, delimiter):
    parts = text.split(delimiter)
    if len(parts) > 1:
        return ' '.join(parts)
    else:
        return str(text)

def remove_bracketed_words(text):
    cleaned_text = re.sub(r'\[.*?\]', '', text)
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
    return cleaned_text

def remove_special_characters(text):
    cleaned_text = re.sub(r'[^a-zA-Z0-9.\s]', '', text)
    return cleaned_text

def lowercase_text(text):
    return text.lower()

In [10]:
data_train['combined_sent'] = data_train['combined_sent'].apply(lambda x: split_string(x, '/'))
data_train['combined_sent'] = data_train['combined_sent'].apply(remove_bracketed_words)
data_train['combined_sent'] = data_train['combined_sent'].apply(remove_special_characters)
data_train['combined_sent'] = data_train['combined_sent'].apply(lowercase_text)

In [11]:
data_train['combined_sent'][0]

'labs were notable for cr 1.7 baseline 0.5 per old records and lactate 2.4. patient has elevated cr'

In [12]:
data_val['combined_sent'] = data_val['combined_sent'].apply(lambda x: split_string(x, '/'))
data_val['combined_sent'] = data_val['combined_sent'].apply(remove_bracketed_words)
data_val['combined_sent'] = data_val['combined_sent'].apply(remove_special_characters)
data_val['combined_sent'] = data_val['combined_sent'].apply(lowercase_text)

In [13]:
data_val['combined_sent'][0]

'no history of blood clots or dvts has never had chest pain prior to one week ago. patient has angina'

In [14]:
data_test['combined_sent'] = data_test['combined_sent'].apply(lambda x: split_string(x, '/'))
data_test['combined_sent'] = data_test['combined_sent'].apply(remove_bracketed_words)
data_test['combined_sent'] = data_test['combined_sent'].apply(remove_special_characters)
data_test['combined_sent'] = data_test['combined_sent'].apply(lowercase_text)

In [15]:
data_test['combined_sent'][0]

'in the ed initial vs revealed t 98.9 hr 73 bp 121 90 rr 15 o2 sat 98 on ra. the patient is hemodynamically stable'

In [16]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

data_train['gold_label'] = le.fit_transform(data_train['gold_label'])
data_train['gold_label'] = data_train['gold_label'].astype('int64')

data_val['gold_label'] = le.fit_transform(data_val['gold_label'])
data_val['gold_label'] = data_val['gold_label'].astype('int64')

data_test['gold_label'] = le.fit_transform(data_test['gold_label'])
data_test['gold_label'] = data_test['gold_label'].astype('int64')


In [17]:
!pip install transformers

In [18]:
import time
import torch
import torch.nn.functional as F
import torchtext

import transformers
from transformers import BertTokenizer, BertForSequenceClassification

In [19]:
torch.backends.cudnn.deterministic = True
RANDOM_SEED = 123
torch.manual_seed(RANDOM_SEED)
DEVICE = torch.device('cuda')

NUM_EPOCHS = 3

In [20]:
train_texts = data_train.iloc[:]['combined_sent'].values
train_labels = data_train.iloc[:]['gold_label'].values

valid_texts = data_val.iloc[:]['combined_sent'].values
valid_labels = data_val.iloc[:]['gold_label'].values


test_texts = data_test.iloc[:]['combined_sent'].values
test_labels = data_test.iloc[:]['gold_label'].values

In [21]:
tokenizer = BertTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True)
valid_encodings = tokenizer(list(valid_texts), truncation=True, padding=True)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [22]:
class MedNLIDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


train_dataset = MedNLIDataset(train_encodings, train_labels)
valid_dataset = MedNLIDataset(valid_encodings, valid_labels)
test_dataset = MedNLIDataset(test_encodings, test_labels)

In [23]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=16, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=False)

In [24]:
model = BertForSequenceClassification.from_pretrained("emilyalsentzer/Bio_ClinicalBERT", num_labels=3)
model.to(DEVICE)
model.train()

optim = torch.optim.AdamW(model.parameters(), lr= 5e-5)


Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model

In [25]:
def compute_accuracy(model, data_loader, device):
    pl = []
    tl = []
    with torch.no_grad():

        for batch_idx, batch in enumerate(data_loader):

            ### Prepare data
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss, logits = outputs['loss'], outputs['logits']

            _, predicted_labels = torch.max(logits, 1)

            pl.extend(predicted_labels.tolist())
            tl.extend(labels.tolist())

    pl = torch.tensor(pl).numpy()
    tl = torch.tensor(tl).numpy()
    return classification_report(tl, pl)

In [26]:
from tqdm import tqdm
start_time = time.time()

for epoch in tqdm(range(NUM_EPOCHS)):

    model.train()

    for batch_idx, batch in enumerate(train_loader):

        ### Prepare data
        input_ids = batch['input_ids'].to(DEVICE)
        attention_mask = batch['attention_mask'].to(DEVICE)
        labels = batch['labels'].to(DEVICE)

        ### Forward
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss, logits = outputs['loss'], outputs['logits']

        ### Backward
        optim.zero_grad()
        loss.backward()
        optim.step()

        ### Logging
        if not batch_idx % 30:
            print (f'Epoch: {epoch+1:04d}/{NUM_EPOCHS:04d} | '
                   f'Batch {batch_idx:04d}/{len(train_loader):04d} | '
                   f'Loss: {loss:.4f}')

    model.eval()

    with torch.set_grad_enabled(False):
        print(f'training report: \n'
              f'{compute_accuracy(model, train_loader, DEVICE)}'
              f'\nvalid accuracy: \n'
              f'{compute_accuracy(model, valid_loader, DEVICE)}')

    print(f'Time elapsed: {(time.time() - start_time)/60:.2f} min')

print(f'Total Training Time: {(time.time() - start_time)/60:.2f} min')
print(f'Test report: {compute_accuracy(model, test_loader, DEVICE)}')

  0%|          | 0/3 [00:00<?, ?it/s]

Epoch: 0001/0003 | Batch 0000/0702 | Loss: 1.2692
Epoch: 0001/0003 | Batch 0030/0702 | Loss: 1.0024
Epoch: 0001/0003 | Batch 0060/0702 | Loss: 0.9179
Epoch: 0001/0003 | Batch 0090/0702 | Loss: 0.5893
Epoch: 0001/0003 | Batch 0120/0702 | Loss: 0.8507
Epoch: 0001/0003 | Batch 0150/0702 | Loss: 0.8894
Epoch: 0001/0003 | Batch 0180/0702 | Loss: 0.7622
Epoch: 0001/0003 | Batch 0210/0702 | Loss: 0.9161
Epoch: 0001/0003 | Batch 0240/0702 | Loss: 1.0707
Epoch: 0001/0003 | Batch 0270/0702 | Loss: 0.3689
Epoch: 0001/0003 | Batch 0300/0702 | Loss: 0.4242
Epoch: 0001/0003 | Batch 0330/0702 | Loss: 0.6111
Epoch: 0001/0003 | Batch 0360/0702 | Loss: 0.6887
Epoch: 0001/0003 | Batch 0390/0702 | Loss: 0.7622
Epoch: 0001/0003 | Batch 0420/0702 | Loss: 0.5202
Epoch: 0001/0003 | Batch 0450/0702 | Loss: 0.7487
Epoch: 0001/0003 | Batch 0480/0702 | Loss: 0.7589
Epoch: 0001/0003 | Batch 0510/0702 | Loss: 0.3677
Epoch: 0001/0003 | Batch 0540/0702 | Loss: 0.6791
Epoch: 0001/0003 | Batch 0570/0702 | Loss: 0.8453


 33%|███▎      | 1/3 [12:31<25:03, 751.61s/it]

training report: 
              precision    recall  f1-score   support

           0       0.86      0.86      0.86      3744
           1       0.77      0.80      0.79      3744
           2       0.81      0.78      0.79      3744

    accuracy                           0.81     11232
   macro avg       0.81      0.81      0.81     11232
weighted avg       0.81      0.81      0.81     11232

valid accuracy: 
              precision    recall  f1-score   support

           0       0.82      0.83      0.83       465
           1       0.72      0.75      0.73       465
           2       0.76      0.72      0.74       465

    accuracy                           0.77      1395
   macro avg       0.77      0.77      0.77      1395
weighted avg       0.77      0.77      0.77      1395

Time elapsed: 12.53 min
Epoch: 0002/0003 | Batch 0000/0702 | Loss: 0.3940
Epoch: 0002/0003 | Batch 0030/0702 | Loss: 0.4045
Epoch: 0002/0003 | Batch 0060/0702 | Loss: 0.7702
Epoch: 0002/0003 | Batch 0090

 67%|██████▋   | 2/3 [25:03<12:31, 751.66s/it]

training report: 
              precision    recall  f1-score   support

           0       0.92      0.95      0.93      3744
           1       0.90      0.86      0.88      3744
           2       0.89      0.90      0.90      3744

    accuracy                           0.90     11232
   macro avg       0.90      0.90      0.90     11232
weighted avg       0.90      0.90      0.90     11232

valid accuracy: 
              precision    recall  f1-score   support

           0       0.82      0.90      0.86       465
           1       0.80      0.71      0.76       465
           2       0.77      0.79      0.78       465

    accuracy                           0.80      1395
   macro avg       0.80      0.80      0.80      1395
weighted avg       0.80      0.80      0.80      1395

Time elapsed: 25.06 min
Epoch: 0003/0003 | Batch 0000/0702 | Loss: 0.4283
Epoch: 0003/0003 | Batch 0030/0702 | Loss: 0.2561
Epoch: 0003/0003 | Batch 0060/0702 | Loss: 0.3645
Epoch: 0003/0003 | Batch 0090

100%|██████████| 3/3 [37:35<00:00, 751.96s/it]

training report: 
              precision    recall  f1-score   support

           0       0.94      0.97      0.96      3744
           1       0.94      0.93      0.93      3744
           2       0.96      0.94      0.95      3744

    accuracy                           0.95     11232
   macro avg       0.95      0.95      0.95     11232
weighted avg       0.95      0.95      0.95     11232

valid accuracy: 
              precision    recall  f1-score   support

           0       0.82      0.88      0.85       465
           1       0.73      0.72      0.72       465
           2       0.79      0.74      0.76       465

    accuracy                           0.78      1395
   macro avg       0.78      0.78      0.78      1395
weighted avg       0.78      0.78      0.78      1395

Time elapsed: 37.60 min
Total Training Time: 37.60 min


Test report:               precision    recall  f1-score   support

           0       0.83      0.88      0.85       474
           1       0.73      0.73      0.73       474
           2       0.78      0.74      0.76       474

    accuracy                           0.78      1422
   macro avg       0.78      0.78      0.78      1422
weighted avg       0.78      0.78      0.78      1422

